## 1

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import math
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

In [2]:
df = pd.read_csv("data.csv")
df.dropna(inplace=True)
df.head()

,date,passengers
0,1949-01,112.0
1,1949-02,118.0
2,1949-03,132.0
3,1949-04,129.0
4,1949-05,121.0


In [3]:
df['date'] = pd.to_datetime(df['date'])
df['month_number'] = (df['date'].dt.year - 1949) * 12 + df['date'].dt.month
df.head()

,date,passengers,month_number
0,1949-01-01,112.0,1
1,1949-02-01,118.0,2
2,1949-03-01,132.0,3
3,1949-04-01,129.0,4
4,1949-05-01,121.0,5


In [81]:
x = df[['month_number']].values
y = df['passengers'].values

X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=100, shuffle=False)

model_linear = LinearRegression()
model_linear.fit(X_train, y_train)
y_pred_linear = model_linear.predict(X_test)

model_poly2 = make_pipeline(PolynomialFeatures(2), LinearRegression())
model_poly2.fit(X_train, y_train)
y_pred_poly2 = model_poly2.predict(X_test)

In [82]:
rmse_poly2 =  root_mean_squared_error(y_test, y_pred_poly2)

{"RMSE polinomiale ": rmse_poly2}

{'RMSE polinomiale ': 68.58918060996757}

In [76]:
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go

In [78]:
x_range = np.linspace(x.min(), x.max(), 200).reshape(-1, 1)
y_fit = model_poly2.predict(x_range)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x.flatten(),
    y=y,
    mode='markers',
    name='Dati originali',
    marker=dict(color='blue', size=8, opacity=0.7)
))

fig.add_trace(go.Scatter(
    x=x_range.flatten(),
    y=y_fit,
    mode='lines',
    name='Regressione polinomiale (grado 2)',
    line=dict(color='red', width=3)
))

fig.update_layout(
    title='Scatter Plot con Regressione Polinomiale',
    xaxis_title='Month Number',
    yaxis_title='Passengers',
    showlegend=True,
    height=500
)

fig.show()

## 2

In [83]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import numpy as np
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [84]:
np.random.seed(0)
x = np.linspace(-3, 3, 100)
x[:10]

array([-3.        , -2.93939394, -2.87878788, -2.81818182, -2.75757576,
       -2.6969697 , -2.63636364, -2.57575758, -2.51515152, -2.45454545])

In [85]:
y = x**3 - x + np.random.normal(scale=3, size=x.shape)
y[:10]

array([-18.70784296, -21.25660604, -18.04272141, -12.84155782,
       -12.60897419, -19.85166557, -12.83718779, -14.96724746,
       -13.70532119, -11.10178827])

In [86]:
app = dash.Dash(__name__)
app.title = "Regressione Polinomiale Interattiva"

app.layout = html.Div([
    html.H2("Regressione Polinomiale Interattiva"),
    dcc.Slider(
        id='poly-degree',
        min=1, max=10, step=1, value=3,
        marks={i: str(i) for i in range(1, 11)},
    ),
    dcc.Graph(id='poly-plot')
])

@app.callback(
    Output('poly-plot', 'figure'),
    [Input('poly-degree', 'value')]
)
def update_graph(degree):
    poly = PolynomialFeatures(degree)
    X_poly = poly.fit_transform(x.reshape(-1, 1))
    model = LinearRegression().fit(X_poly, y)
    y_pred = model.predict(X_poly)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Dati'))
    fig.add_trace(go.Scatter(x=x, y=y_pred, mode='lines', name=f'Fit grado {degree}'))
    fig.update_layout(title=f"Regressione di grado {degree}",
                      xaxis_title="x", yaxis_title="y")
    return fig

if __name__ == '__main__':
    app.run(debug=True)